<a href="https://colab.research.google.com/github/MigguuelT/Miguel-Torikachvili/blob/main/Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar as redes sociais da Alura:** Fique de olho nos perfis da Alura no [YouTube](https://www.youtube.com/@AluraCursosOnline), [LinkedIn](https://www.linkedin.com/company/aluraonline/), [Instagram](https://www.instagram.com/aluraonline/), [Twitter](https://twitter.com/AluraOnline), e [Facebook](https://www.facebook.com/AluraCursosOnline).
*   **Verificar a página de Imersões da Alura:** Consulte a página de Imersões da Alura para ver se há alguma imersão programada ou se existe a possibilidade de se inscrever para receber notificações sobre futuras edições.
*   **Entrar na comunidade da Alura:** Participe da comunidade da Alura para ficar por dentro das novidades e eventos.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config = {"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura já aconteceu entre os dias 12 e 16 de maio de 2025. As inscrições para esta edição foram até o dia 11 de maio de 2025.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA com Google Gemini Alura', 'Alura imersão IA Google Gemini']
Páginas utilizadas na resposta: starten.tech, tecmundo.com.br, youtube.com, alura.com.br



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [21]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [22]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [23]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [24]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: data science
Maravilha! Vamos então criar o post sobre novidades em data science

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes e recentes em data science, vou realizar algumas buscas no Google focando em notícias, novidades e tendências do último mês. O objetivo é identificar 5 lançamentos que geraram grande impacto ou entusiasmo na comunidade.
> 
> 
> Com base nas minhas buscas, aqui estão 5 lançamentos e tendências relevantes em data science para maio de 2025:
> 
> 1.  **Aumento do uso de IA Generativa:** A inteligência artificial generativa, impulsionada por modelos como GPT, está se tornando fundamental para automatizar processos, criar conteúdo e gerar insights valiosos. Ferramentas como o ChatGPT estão sendo usadas para gerar snippets de código e obter insights rápidos, que são então refinados manualmente por cientistas de dados. A integração de IA e aprendizado de máquina em fluxos de trabalho de ciência de dados está transformando o campo, com ferramentas de AutoML (Automated Machine Learning) facilitando a construção e implementação de modelos. Empresas planejam usar IA para substituir tarefas de nível inferior até o final de 2024, embora haja preocupações sobre a superdependência da IA.
> 2.  **Big Data e Analytics em Tempo Real:** O volume de dados continua a crescer exponencialmente, com a criação global de dados atingindo níveis astronômicos. A análise em tempo real tornou-se uma necessidade competitiva, com um crescimento de mercado projetado de cerca de 23,8% CAGR até 2028. Empresas agora esperam obter insights desses conjuntos de dados massivos e em rápida evolução, buscando dashboards de streaming que são atualizados a cada segundo.
> 3.  **Edge Computing e IA:** O crescimento da Internet das Coisas (IoT) está impulsionando a ciência de dados em direção à borda (edge). O Edge AI permite o processamento de dados em tempo real na fonte, reduzindo a latência e os custos de largura de banda. Essa tendência é fundamental para aplicações em cidades inteligentes, automação industrial e saúde, onde insights oportunos podem impulsionar ações imediatas. Em 2025, espera-se um aumento no uso de modelos de machine learning executados em dispositivos periféricos, como sensores e câmeras inteligentes.
> 4.  **Cursos e Certificações em Data Science:** Há uma crescente demanda por profissionais qualificados em data science, com diversas opções de cursos e certificações disponíveis. DataCamp e IBM Data Science Professional Certificate são algumas das opções populares. Habilidades técnicas importantes incluem Python e R para extração de dados, SQL e PostgreSQL para manipulação de bancos de dados, e ferramentas de visualização como Power BI, Tableau e Google Data Studio.
> 5.  **Data Science e Sustentabilidade:** A análise de dados está sendo utilizada para abordar problemas globais, como as mudanças climáticas e a gestão de recursos. Empresas estão priorizando a ética e a privacidade em suas estratégias de Data Science, adotando técnicas de anonimização e melhorando a transparência com os usuários, devido ao aumento da regulamentação em torno dos dados pessoais. A sustentabilidade também está se tornando um tema importante, com empresas buscando usar dados para otimizar o uso de recursos e reduzir o impacto ambiental.
> 
> Essas tendências refletem um cenário dinâmico em data science, com um foco crescente na aplicação de IA, análise de dados em tempo real, computação de borda e ética de dados.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Ok, com base nos lançamentos e tendências de Data Science para maio de 2025 que você identificou, vou elaborar um plano para um post de mídia social.
> 
> **Primeiramente, algumas pesquisas para aprofundar o conhecimento sobre os temas:**
> 
> 
> Com base nas pesquisas e nos lançamentos iniciais, o tema mais relevante para um post de mídia social é:
> 
> **1. Aumento do uso de IA Generativa em Data Science:**
> 
> *   **Por que é relevante:** A IA Generativa está transformando a forma como os cientistas de dados trabalham, automatizando tarefas, acelerando a obtenção de insights e democratizando o acesso aos dados. É uma tendência que está impactando todas as áreas, desde a geração de código até a análise preditiva.
> *   **Pontos mais relevantes:**
>     *   Automação de tarefas repetitivas e geração de código.
>     *   Democratização do acesso à análise de dados para não especialistas.
>     *   Aplicações práticas em áreas como marketing, vendas e finanças.
>     *   Integração multimodal (texto, imagem, vídeo, áudio) para análises mais ricas.
>     *   Preocupações com governança e qualidade dos dados.
> 
> **Plano para o post:**
> 
> *   **Título:** IA Generativa: A Revolução do Data Science em 2025! 🚀
> *   **Imagem/Vídeo:** Um visual que mostre um cientista de dados utilizando uma ferramenta de IA Generativa para analisar dados de forma eficiente.
> *   **Texto:**
> 
>     *   **Introdução:** "A IA Generativa chegou para transformar o Data Science! Descubra como essa tecnologia está automatizando tarefas, acelerando insights e democratizando o acesso aos dados em 2025."
>     *   **Tópicos:**
>         *   **Automação e Eficiência:** "Diga adeus às tarefas repetitivas! A IA Generativa automatiza a criação de código, relatórios e visualizações, liberando os cientistas de dados para se concentrarem em análises estratégicas."
>         *   **Democratização do Acesso:** "Análise de dados para todos! A IA Generativa permite que profissionais sem conhecimento técnico façam perguntas complexas e obtenham respostas rápidas, democratizando o acesso aos insights."
>         *   **Aplicações Práticas:** "Deixe a IA Generativa gerar automaticamente relatórios narrativos a partir de dados brutos e criar visualizações personalizadas sem necessidade de programação. Aplicações em marketing, vendas, finanças e muito mais!"
>         *   **O Futuro é Multimodal:** "Prepare-se para análises mais ricas! A IA Generativa está evoluindo para integrar texto, imagem, vídeo e áudio, permitindo insights mais completos e contextualizados."
>         *   **Atenção à Qualidade:** "Com grandes poderes vêm grandes responsabilidades! É crucial garantir a governança e a qualidade dos dados para evitar vieses e obter resultados confiáveis."
>     *   **Chamada para ação:** "Quer saber mais sobre como a IA Generativa pode impulsionar sua estratégia de dados? Deixe um comentário com suas dúvidas e compartilhe este post com seus colegas!"
> *   **Hashtags:** #DataScience #IAGenerativa #InteligenciaArtificial #MachineLearning #AI #Inovacao #Tendencias2025 #BigData #Analytics
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## Rascunho de Post para Instagram
> 
> **[Imagem/Vídeo: Um cientista de dados utilizando uma ferramenta de IA Generativa para analisar dados de forma eficiente.]**
> 
> 🚀 IA Generativa: A Revolução do Data Science em 2025! 🚀
> 
> A IA Generativa chegou para transformar o Data Science! Descubra como essa tecnologia está automatizando tarefas, acelerando insights e democratizando o acesso aos dados.
> 
> ✨ Automação e Eficiência: Diga adeus às tarefas repetitivas! A IA Generativa automatiza a criação de código, relatórios e visualizações, liberando os cientistas de dados para análises estratégicas.
> 
> 📊 Democratização do Acesso: Análise de dados para todos! A IA Generativa permite que profissionais sem conhecimento técnico façam perguntas complexas e obtenham respostas rápidas, democratizando o acesso aos insights.
> 
> 💡 Aplicações Práticas: Deixe a IA Generativa gerar automaticamente relatórios narrativos a partir de dados brutos e criar visualizações personalizadas sem necessidade de programação. Aplicações em marketing, vendas, finanças e muito mais!
> 
> 🌐 O Futuro é Multimodal: Prepare-se para análises mais ricas! A IA Generativa está evoluindo para integrar texto, imagem, vídeo e áudio, permitindo insights mais completos e contextualizados.
> 
> ⚠️ Atenção à Qualidade: Com grandes poderes vêm grandes responsabilidades! É crucial garantir a governança e a qualidade dos dados para evitar vieses e obter resultados confiáveis.
> 
> 🚀 Quer saber mais sobre como a IA Generativa pode impulsionar sua estratégia de dados? Deixe um comentário com suas dúvidas e compartilhe este post com seus colegas!
> 
> #DataScience #IAGenerativa #Inovacao #Alura
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
